In [2]:
# Import some packages
library(reshape2)
library(ggplot2)
library(cowplot)
library(ez)
library(nlme)
library(multcomp)
library(pastecs)
theme_set(theme_cowplot(font_size=8))
library(usdm)
library(r2glmm)
library(car)
library(MASS)
library(MCMCglmm)
library(lmerTest)
library(expss)
library(moments)
library(MuMIn)
library(margins)
library(dplyr)

In [3]:
# Read signals, extract directions and distance thresholds
signals = read.csv("../metrics/all_intervals_all_data.csv")
colnames(signals)

# Asign direction columns for acceleration and jerk (positive for >= 0 and negative for < 0)
signals$pos_latA <- signals$MaxLatAcc >= 0
signals$pos_longA <- signals$MaxLongAcc >= 0

signals$pos_latJ <- signals$MaxLatJerk >= 0
signals$pos_longJ <- signals$MaxLongJerk >= 0

# Thresholds for the presence of a lead vehicle
signals$present_stop <- signals$LongDist < 190
signals$present_pass <- signals$LatDist < 190

[1] "X"            "pid"          "trust"        "knowledge"    "age"         
 [6] "LatDist"      "LongDist"     "MaxVel"       "MeanVel"      "MaxLatAcc"   
[11] "MeanLatAcc"   "MaxLongAcc"   "MeanLongAcc"  "MaxLatJerk"   "MeanLatJerk" 
[16] "MaxLongJerk"  "MeanLongJerk" "NumPks"       "MaxPkAmp"     "MeanScl"     
[21] "MeanEntropy"  "MaxEntropy"   "MeanDwell"    "MaxDwell"     "MeanHr"      
[26] "MaxHr"        "Trial"        "Time"

In [4]:
# Max-valued regressions

# column indexes for the response/dependent variables of interest (NumPks, MaxPkAmp, MaxHR -- MaxEntropy is
# handled separately due to missing data in signals)
response_ids = c(18, 19, 26)
responses = colnames(signals)[response_ids]

#---------------------------------------- Models for all main effects --------------------------------------#
main_max = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ Time +
                    present_stop +
                    MaxLongAcc*pos_longA + MaxLatAcc*pos_latA + MaxLongJerk*pos_longJ + MaxLatJerk*pos_latJ +
                    present_pass +
                    age + knowledge + trust,
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        correlation = corAR1(0, form=~1|pid/Trial),
        data = signals,
        method = "ML"
    )
})

entropy_main_max = lme(
    MaxEntropy ~ Time +
                present_stop +
                MaxLongAcc*pos_longA + MaxLatAcc*pos_latA + MaxLongJerk*pos_longJ + MaxLatJerk*pos_latJ +
                present_pass +
                age + knowledge + trust,
    
    random = ~1|pid/Trial,
    correlation = corAR1(0, form=~1|pid/Trial),
    data = na.omit(signals),
    method = "ML"
)

#------------------------------------ Models for all interaction effects ------------------------------------#
inter_max = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ Time +
                    present_stop +
                    MaxLongAcc*pos_longA + MaxLatAcc*pos_latA + MaxLongJerk*pos_longJ + MaxLatJerk*pos_latJ +
                    present_stop*MaxLongAcc*pos_longA + present_stop*MaxLongJerk*pos_longJ +
                    present_pass +
                    age + knowledge + trust,
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        correlation = corAR1(0, form=~1|pid/Trial),
        data = signals,
        method = "ML"
    )
})

entropy_inter_max = lme(
    MaxEntropy ~ Time +
                present_stop +
                MaxLongAcc*pos_longA + MaxLatAcc*pos_latA + MaxLongJerk*pos_longJ + MaxLatJerk*pos_latJ +
                present_stop*MaxLongAcc*pos_longA + present_stop*MaxLongJerk*pos_longJ +
                present_pass +
                age + knowledge + trust,
    
    random = ~1|pid/Trial,
    correlation = corAR1(0, form=~1|pid/Trial),
    data = na.omit(signals),
    method = "ML"
)

In [5]:
# Mean-valued regressions

# column indexes for the response/dependent variables of interest (Mean SCL and Mean HR -- MeanEntropy is
# handled separately due to missing data in signals
response_ids = c(20, 25)
responses = colnames(signals)[response_ids]

#---------------------------------------- Models for all main effects --------------------------------------#
main_mean = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ Time + 
                    present_stop + 
                    MeanLongAcc + MeanLatAcc + MeanLongJerk + MeanLatJerk + 
                    present_pass + 
                    age + knowledge + trust,
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        correlation = corAR1(0, form=~1|pid/Trial),
        data = signals,
        method = "ML"
    )
})

entropy_main_mean = lme(
    MeanEntropy ~ Time + 
                present_stop + 
                MeanLongAcc + MeanLatAcc + MeanLongJerk + MeanLatJerk + 
                present_pass + 
                age + knowledge + trust,
    
    random = ~1|pid/Trial,
    correlation = corAR1(0, form=~1|pid/Trial),
    data = na.omit(signals),
    method = "ML"
)

#------------------------------------ Models for all interaction effects ------------------------------------#
inter_mean = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ Time + 
                    present_stop + 
                    MeanLongAcc + MeanLatAcc + MeanLongJerk + MeanLatJerk +
                    present_stop:MeanLongAcc + present_stop:MeanLongJerk +
                    present_pass + 
                    age + knowledge + trust,
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        correlation = corAR1(0, form=~1|pid/Trial),
        data = signals,
        method = "ML"
    )
})

entropy_inter_mean = lme(
    MeanEntropy ~ Time + 
                present_stop + 
                MeanLongAcc + MeanLatAcc + MeanLongJerk + MeanLatJerk + 
                present_stop:MeanLongAcc + present_stop:MeanLongJerk +
                present_pass + 
                age + knowledge + trust,
    
    random = ~1|pid/Trial,
    correlation = corAR1(0, form=~1|pid/Trial),
    data = na.omit(signals),
    method = "ML"
)

In [6]:
# Combine all main effect models and interaction effect models (excluding entropy which we process separately)
main_eff_models = c(main_max, main_mean)
inter_eff_models = c(inter_max, inter_mean)

response_names = c("NUMBER OF PEAKS", "MAX PEAK AMPLITUDE", "MAX HR", "MEAN SCL", "MEAN HR")
model_idx  = c(1:5)

In [7]:
cat("ALL MAIN EFFECTS\n")
cat("---------------------------------------------------------------------------------------------------------\n")

model_print = function(x, y, z){
    cat(y[x])
    cat("\n")
    print(summary(z[[x]]))
    cat("\n\n\n")
    cat("=====================================================================================================
    \n\n\n")
}

lapply(model_idx, model_print, y=response_names, z=main_eff_models)

cat("MAXIMUM ENTROPY\n")
print(summary(entropy_main_max))
cat("\n=====================================================================================================
\n\n\n")

cat("MEAN ENTROPY\n")
print(summary(entropy_main_mean))
cat("=====================================================================================================\n")

ALL MAIN EFFECTS
---------------------------------------------------------------------------------------------------------
NUMBER OF PEAKS
Linear mixed-effects model fit by maximum likelihood
 Data: signals 
       AIC      BIC    logLik
  3199.826 3324.909 -1576.913

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:   0.1440816

 Formula: ~1 | Trial %in% pid
        (Intercept)  Residual
StdDev:  0.04830922 0.6041696

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
        Phi 
-0.03796686 
Fixed effects: eval(substitute(resp ~ Time + present_stop + MaxLongAcc * pos_longA +      MaxLatAcc * pos_latA + MaxLongJerk * pos_longJ + MaxLatJerk *      pos_latJ + present_pass + age + knowledge + trust, list(resp = as.name(x)))) 
                               Value  Std.Error   DF   t-value p-value
(Intercept)                0.4453303 0.23241944 1606  1.916063  0.0555
Time                      -0.0010245 0.00361010 1606 -0.283798  0.7766
present_sto

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

MAXIMUM ENTROPY
Linear mixed-effects model fit by maximum likelihood
 Data: na.omit(signals) 
        AIC       BIC   logLik
  -5341.413 -5221.598 2693.707

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:    0.025424

 Formula: ~1 | Trial %in% pid
         (Intercept)   Residual
StdDev: 2.752198e-06 0.04483558

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
      Phi 
0.6983421 
Fixed effects: MaxEntropy ~ Time + present_stop + MaxLongAcc * pos_longA + MaxLatAcc *      pos_latA + MaxLongJerk * pos_longJ + MaxLatJerk * pos_latJ +      present_pass + age + knowledge + trust 
                                Value  Std.Error   DF   t-value p-value
(Intercept)                0.30423297 0.03806436 1274  7.992593  0.0000
Time                      -0.00081405 0.00038272 1274 -2.127029  0.0336
present_stopTRUE           0.00889840 0.00448398 1274  1.984487  0.0474
MaxLongAcc                -0.00134476 0.00240335 1274 -0.559535  0.5759
pos_longATRUE

In [8]:
cat("ALL INTERACTION EFFECTS\n")
cat("---------------------------------------------------------------------------------------------------------\n")

model_print = function(x, y, z){
    cat(y[x])
    cat("\n")
    print(summary(z[[x]]))
    cat("\n\n\n")
    cat("=====================================================================================================
    \n\n\n")
}

lapply(model_idx, model_print, y=response_names, z=inter_eff_models)

cat("MAXIMUM ENTROPY\n")
print(summary(entropy_inter_max))
cat("\n=====================================================================================================
\n\n\n")

cat("MEAN ENTROPY\n")
print(summary(entropy_inter_mean))
cat("=====================================================================================================\n")

ALL INTERACTION EFFECTS
---------------------------------------------------------------------------------------------------------
NUMBER OF PEAKS
Linear mixed-effects model fit by maximum likelihood
 Data: signals 
       AIC      BIC    logLik
  3150.967 3308.681 -1546.484

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:   0.1429634

 Formula: ~1 | Trial %in% pid
        (Intercept)  Residual
StdDev:  0.02718916 0.5944352

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
        Phi 
-0.04083571 
Fixed effects: eval(substitute(resp ~ Time + present_stop + MaxLongAcc * pos_longA +      MaxLatAcc * pos_latA + MaxLongJerk * pos_longJ + MaxLatJerk *      pos_latJ + present_stop * MaxLongAcc * pos_longA + present_stop *      MaxLongJerk * pos_longJ + present_pass + age + knowledge +      trust, list(resp = as.name(x)))) 
                                                Value Std.Error   DF   t-value
(Intercept)                                 0.5

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

MAXIMUM ENTROPY
Linear mixed-effects model fit by maximum likelihood
 Data: na.omit(signals) 
        AIC       BIC   logLik
  -5336.365 -5185.294 2697.182

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:  0.02544855

 Formula: ~1 | Trial %in% pid
         (Intercept)   Residual
StdDev: 2.848994e-06 0.04473421

Correlation Structure: AR(1)
 Formula: ~1 | pid/Trial 
 Parameter estimate(s):
      Phi 
0.6986038 
Fixed effects: MaxEntropy ~ Time + present_stop + MaxLongAcc * pos_longA + MaxLatAcc *      pos_latA + MaxLongJerk * pos_longJ + MaxLatJerk * pos_latJ +      present_stop * MaxLongAcc * pos_longA + present_stop * MaxLongJerk *      pos_longJ + present_pass + age + knowledge + trust 
                                                 Value  Std.Error   DF
(Intercept)                                 0.30587966 0.03817953 1268
Time                                       -0.00083721 0.00038539 1268
present_stopTRUE                           -0.02836695 0.02260438 1268
Max